In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import math
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from helper_functions import *

Load data

In [2]:
# Create headers
index_col_names=['unit_id','time_cycle']
operat_set_col_names=['op_conditions_{}'.format(i) for i in range(1,4)]
sensor_measure_col_names=['sensor_{}'.format(i) for i in range(1,22)]
all_col=index_col_names+operat_set_col_names+sensor_measure_col_names
print(all_col)

['unit_id', 'time_cycle', 'op_conditions_1', 'op_conditions_2', 'op_conditions_3', 'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10', 'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15', 'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20', 'sensor_21']


In [3]:
# Import Data - NASA dataset 260 train trajectories, 259 tests and 6 conditions
raw_train_df=pd.read_csv('Data/train_FD002.txt',delim_whitespace=True,names=all_col)
raw_train_df.head()

/var/folders/gm/pvnntq692jd5ng0rtk5l0z9m0000gn/T/ipykernel_57098/230756806.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  raw_train_df=pd.read_csv('Data/train_FD002.txt',delim_whitespace=True,names=all_col)


,unit_id,time_cycle,op_conditions_1,op_conditions_2,op_conditions_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,...,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,...,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,...,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,...,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,...,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


K-means clustering model.

*   Develop a k-means clustering algorithm to find clusters.
*   Save model and centroids.

*   Use model to predict the cluster every data point belongs to.





In [4]:
op_condts = raw_train_df[['op_conditions_1'	, 'op_conditions_2', 'op_conditions_3']]
op_condts.shape

(53759, 3)

In [5]:
# Use K-means to find clusters
kmeans = KMeans(n_clusters = 6) # Greedy k-means is used when the n_clusters is not specified, it enables faster convergence
kmeans.fit(op_condts)
cluster_labels_2 = kmeans.labels_
centroids = kmeans.cluster_centers_

In [6]:
centroids

array([[4.20029763e+01, 8.40485184e-01, 1.00000000e+02],
       [2.00029990e+01, 7.00515193e-01, 1.00000000e+02],
       [2.50030380e+01, 6.20501875e-01, 6.00000000e+01],
       [1.50451268e-03, 4.94343610e-04, 1.00000000e+02],
       [1.00029713e+01, 2.50495035e-01, 1.00000000e+02],
       [3.50030490e+01, 8.40500585e-01, 1.00000000e+02]])

In [7]:
cluster_labels_2

array([5, 0, 2, ..., 2, 2, 5], dtype=int32)

Save Kmeans model

In [8]:
# Save Model Using joblib
import joblib
# save the model to disk
filename = 'kmeans_model'
joblib.dump(kmeans, filename)


['kmeans_model']

Use Model to predict clusters of other NASA datasets

In [9]:
loaded_model = joblib.load('kmeans_model')

In [10]:
# Import Data - NASA dataset 260 train trajectories, 259 tests and 6 conditions
raw_test_df=pd.read_csv('Data/train_FD004.txt',delim_whitespace=True,names=all_col)
raw_test_df.head()

/var/folders/gm/pvnntq692jd5ng0rtk5l0z9m0000gn/T/ipykernel_57098/8867955.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  raw_test_df=pd.read_csv('Data/train_FD004.txt',delim_whitespace=True,names=all_col)


,unit_id,time_cycle,op_conditions_1,op_conditions_2,op_conditions_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,...,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,...,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,...,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,...,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,...,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754


In [11]:
op_condts = raw_test_df[['op_conditions_1'	, 'op_conditions_2', 'op_conditions_3']]
cluster_labels = loaded_model.predict(op_condts)
print(cluster_labels.shape)

(61249,)


In [12]:
np.unique(cluster_labels)

array([0, 1, 2, 3, 4, 5], dtype=int32)